In [ ]:
import pandas as pd
import numpy as np
import math

In [2]:
pd.options.display.float_format = '{:.0f}'.format

<h2>Only for google colab:</h2>

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

# !ls '/content/gdrive/My Drive/Data challenge/code'

# # import single json file as dataframe
# df = pd.read_json('/content/gdrive/My Drive/Data challenge/code/airlines-1558527599826.json', orient = 'records', lines = True)

In [36]:
# df = pd.read_json('airlines-1558527599826.json', orient = 'records', lines = True)
df = pd.read_csv('philipp.csv', usecols=['id_str','created_at','in_reply_to_status_id_str'])

In [38]:
df

,created_at,id_str,in_reply_to_status_id_str
0,Wed May 22 12:20:00 +0000 2019,1131172858951024640,nan
1,Wed May 22 12:20:01 +0000 2019,1131172864147808256,nan
2,Wed May 22 12:20:02 +0000 2019,1131172867985485824,1131032916232826880
3,Wed May 22 12:20:12 +0000 2019,1131172909463027712,nan
4,Wed May 22 12:20:28 +0000 2019,1131172975682605056,nan
...,...,...,...
6454270,Mon Mar 30 18:43:14 +0000 2020,1244696703690772480,nan
6454271,Mon Mar 30 18:43:15 +0000 2020,1244696708983984128,nan
6454272,Mon Mar 30 18:43:15 +0000 2020,1244696710447800320,nan
6454273,Mon Mar 30 18:43:16 +0000 2020,1244696713350217728,nan


In [52]:
df.drop(df[df['id_str'].astype(str).str.len()<5].index, inplace = True)
df['id_str']=df['id_str'].astype('int64')
# df['id']=df['id'].astype('int64')
df = df.rename({'id_str': 'id','in_reply_to_status_id_str':'in_reply_to_status_id'}, axis=1)

In [55]:
df=df.sort_values('created_at')

In [56]:
df.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = True)

In [63]:
df.drop(df[df['id']==df['in_reply_to_status_id']].index,inplace= True)

In [64]:
df_reply=df[~df['in_reply_to_status_id'].isnull()]
index=0
def met2():
    def lookup(ids):
        ret_list=[]
        df_np=df_reply.loc[df_reply['in_reply_to_status_id']==ids]['id'].values
        for row in range(0,len(df_np)):
            global index
            used[index]=df_np[row]
            index+=1
            buffer=df[df['id']==df_np[row]]
            if len(buffer)==0:
                ret_list.append(df_np[row])
            else:
                lookup_ret=lookup(buffer['id'].item())
                if lookup_ret:
                    ret_list.extend([df_np[row],lookup_ret])
                else:
                    ret_list.append(df_np[row])
        if len(ret_list)==0:
            return
        return ret_list
    used=np.zeros(len(df), dtype='int64')
    with open('conversations_list.txt', 'w') as f:
        id_np=np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64'))
        for row in range(0,len(id_np)):
            if math.isnan(id_np[row]):continue
            this_id=id_np[row]
            if this_id in used: continue
            bla=[this_id]
            try:
                lookup_ret=lookup(this_id)
            except:
                continue
            if lookup_ret:
                if len(lookup_ret)==1:
                    bla.append(lookup_ret[0])
                else:
                    bla.extend(lookup_ret)
                f.write("%s\n" % bla)

In [ ]:
%%time
met2()

In [24]:
np.concatenate([np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64')),df[~df['in_reply_to_status_id'].isnull()]['id']])

array([1131173010508062720, 1131173313982734336, 1131173345918148608, ...,
       1177008913675247616, 1177008933992402944, 1177009156261392384],
      dtype=int64)

In [26]:
pd.DataFrame(np.concatenate([np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64')),df[~df['in_reply_to_status_id'].isnull()]['id']])).to_csv('tweets_in_conv.csv')